In [712]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from copy import copy
import pandas_profiling as profiling

In [713]:
file_path ="C:/Users/Andrew Mark/Google Drive/Projects/Kaggle/house prices"
train = pd.read_csv(file_path+"/train.csv")
test_ori = pd.read_csv(file_path+"/test.csv")
sub = pd.read_csv(file_path+"/sample_submission.csv")

In [714]:
test = copy(test_ori)

In [715]:
# train.info()

In [716]:
# profiling.ProfileReport(train)

In [717]:
# profiling.ProfileReport(test)

In [718]:
ids = train['Id'].unique()

In [719]:
#Join train and test
join = train.append(test,ignore_index = True, sort=False)

In [720]:
clm = ['Id','SalePrice']
features = join.drop(clm, axis = 1)

In [721]:
features.drop(['Utilities','Street','Fence','GarageArea','TotRmsAbvGrd', 'Alley', 'FireplaceQu','PoolQC','MiscFeature' ], axis=1, inplace=True)

In [722]:
# #Checking corelation
# sns.heatmap(train.corr(), annot=True)

In [723]:
# profiling.ProfileReport(features)

In [724]:
#Filling missing values
def fill_missing_values(df):
    missing = df.isnull().sum()
    missing = missing[missing > 0]
    for column in list(missing.index):
        if df[column].dtype == 'object':
            df[column].fillna(df[column].value_counts().index[0], inplace=True)
        elif df[column].dtype == 'int64' or 'float64' or 'int16' or 'float16':
            df[column].fillna(df[column].median(), inplace=True)          

In [725]:
#Converting String to Int
def convert (df):
    # Find the columns of object type along with their column index
    object_cols = list(df.select_dtypes(exclude=[np.number]).columns)
    object_cols_ind = []
    for col in object_cols:
        object_cols_ind.append(df.columns.get_loc(col))

    # Encode the categorical columns with numbers    
    label_enc = LabelEncoder()
    for i in object_cols_ind:
        df.iloc[:,i] = label_enc.fit_transform(df.iloc[:,i])

In [726]:
fill_missing_values(features)

In [727]:
convert(features)

In [728]:
features = features.dropna()

In [729]:
#Normalize data before further processing
from sklearn.preprocessing import MinMaxScaler
columns_to_norm = features.columns
features[columns_to_norm] = MinMaxScaler().fit_transform(features[columns_to_norm])

In [730]:
# New Features
features['Total SF'] = features['TotalBsmtSF'] + features['1stFlrSF'] + features['2ndFlrSF']
features['Home Quality'] = features['OverallQual'] + features['OverallCond']
features['Total Bathrooms'] = (features['FullBath'] + (0.5 * features['HalfBath']) +
                               features['BsmtFullBath'] + (0.5 * features['BsmtHalfBath']))
features['Total SQR footage'] = (features['BsmtFinSF1'] + features['BsmtFinSF2'] +
                                 features['1stFlrSF'] + features['2ndFlrSF'])

In [731]:
# Recreate Train and test
train_df = features[join['Id'].isin(ids)]
test_df = features[~join['Id'].isin(ids)]

In [732]:
# Define Train and test data for modeling
X_train = train_df
y_train = np.log(train['SalePrice'])

X_test = test_df
y_test = np.log(sub['SalePrice'])

In [733]:
X_test.shape, y_test.shape

((1459, 74), (1459,))

In [734]:
X_train.shape, y_train.shape

((1460, 74), (1460,))

In [736]:
# training and prediction
import xgboost as XGBRegressor
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

xgb = xgb.XGBRegressor(random_state=53, n_jobs=1, learning_rate=0.01,n_estimators=500, max_depth=4)
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)

rf = RandomForestRegressor(n_estimators = 500, random_state = 53)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)

gbr = GradientBoostingRegressor(learning_rate=0.01,n_estimators = 500, random_state = 53)
gbr.fit(X_train, y_train)
gbr_pred = gbr.predict(X_test)


print("XGBRegressor MSE: \t\t",metrics.mean_squared_error(y_test, xgb_pred))
print("RandomForestRegressor MSE: \t",metrics.mean_squared_error(y_test, rf_pred))
print("GradientBoostingRegressor MSE: ",metrics.mean_squared_error(y_test, gbr_pred))

C:\Users\Andrew Mark\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


[23:03:24] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
XGBRegressor MSE: 		 0.14911590380998124
RandomForestRegressor MSE: 	 0.13406387242751547
GradientBoostingRegressor MSE:  0.13417280622502384


In [ ]:
def rmse (y_test, y_pred):
    return np.sqrt(metrics.mean_squared_error(np.log(y_test),np.log(y_pred))

In [740]:
print ("Model Root Mean Squared Error : ", rmse(y_test, rf_pred))

Model Root Mean Squared Error :  0.36614733704823726


In [741]:
price = []
price = np.floor(np.expm1(rf_pred))

In [742]:
submit = pd.DataFrame({'Id':test['Id'], 'SalePrice':price})
print(submit.head(10))
submit.to_csv('submission_house.csv', index = False)

     Id  SalePrice
0  1461   124800.0
1  1462   154091.0
2  1463   176397.0
3  1464   180926.0
4  1465   190813.0
5  1466   180524.0
6  1467   167172.0
7  1468   172791.0
8  1469   182771.0
9  1470   124405.0
